In [1]:
import numpy as np
import pandas as pd
import glob as gb
import os

In [2]:
sell_path = {}
supply_path = {}
inventory_path = {}

supply = {}
sell = {}
inventory = {}

for i in {'b', 'm', 's'}:
    sell_path[i] = gb.glob(os.getcwd() + '/input/*' + i + '*-sell.csv')
    supply_path[i] = gb.glob(os.getcwd() + '/input/*' + i + '*-supply.csv')
    inventory_path[i] = gb.glob(os.getcwd() + '/input/*' + i + '*-inventory.csv')
    for j in range(0, len(sell_path[i])):
        sell[i + str(j+1)] = pd.read_csv(sell_path[i][j], index_col = 'date', 
                                         parse_dates = True)
        supply[i + str(j+1)] = pd.read_csv(supply_path[i][j], index_col = 'date', 
                                           parse_dates = True)
        inventory[i + str(j+1)] = pd.read_csv(inventory_path[i][j], index_col = 'date', 
                                              parse_dates = True)
    

In [3]:
def sell_upg(sell):
    
    upg_tmp = pd.DataFrame(columns=['sku_num'], index = sell.index)
    upg_tmp['sku_num'] = sell['sku_num'].apply(lambda x: 0 if x.find('ap') == 6 else 1)
    daily = pd.crosstab(upg_tmp.index, upg_tmp['sku_num'], margins = True)
    daily = daily.drop(['All'])
    daily_new = pd.DataFrame(columns=[ 'apple', 'pen'], 
                             index = upg_tmp.index.drop_duplicates())
    daily_new['apple'] = daily[0].values
    daily_new['pen'] = daily[1].values
        
    return daily_new

In [11]:
for i in sell.keys():
    sell[i] = sell_upg(sell[i])

In [87]:
def stolen(sell, supply, inventory):
    
    st = pd.DataFrame(columns=['apple', 'pen'], index = inventory.index)
    sell = sell.resample('M').sum()
    supply = supply.resample('M').sum()
    inv = pd.DataFrame(columns=['apple', 'pen'], index = inventory.index)
    inv['apple'] = inventory['apple'] - inventory['apple'] .shift(1)
    inv.loc[inv.index[0], 'apple'] = inventory.loc[inventory.index[0], 'apple']
    inv['pen'] = inventory['pen'] - inventory['pen'] .shift(1)
    inv.loc[inv.index[0], 'pen'] = inventory.loc[inventory.index[0], 'pen']
    st['apple'] = inv['apple'] + sell['apple'] - supply['apple']
    st['pen'] = inv['pen'] + sell['pen'] - supply['pen'] 
    return st.abs()

            apple   pen
date                   
2006-01-31   10.0  11.0
2006-02-28    6.0   6.0
2006-03-31    7.0   6.0
2006-04-30    6.0  14.0
2006-05-31    8.0   1.0
2006-06-30    7.0   9.0
2006-07-31   10.0  12.0
2006-08-31    3.0   5.0
2006-09-30    6.0   4.0
2006-10-31    1.0   9.0


In [90]:
steal = {}

for i in sell.keys():
    steal[i] = stolen(sell[i], supply[i], inventory[i])
    
print(stolen(sell['b1'], supply['b1'], inventory['b1']).head(n=10))

            apple   pen
date                   
2006-01-31   10.0  11.0
2006-02-28    6.0   6.0
2006-03-31    7.0   6.0
2006-04-30    6.0  14.0
2006-05-31    8.0   1.0
2006-06-30    7.0   9.0
2006-07-31   10.0  12.0
2006-08-31    3.0   5.0
2006-09-30    6.0   4.0
2006-10-31    1.0   9.0
